# EXERCISE 6

In [1]:
import re

from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers.aead import AESCCM
from cryptography.hazmat.primitives.keywrap import aes_key_unwrap

Validation code

In [2]:
def validate_solution(solution, start, end):
    match = re.match(fr'^{start}.*{end}$', solution)
    print('Solution:', solution)
    if match:
        print('\033[92mCorrect Solution.')
    else:
        print('\033[0;31mIncorrect Solution.')
    print('\033[0m')

def check_equals(expected, solution):
    print('Expected:', expected)
    print('Solution:', solution)
    if expected == solution:
        print('\033[92mCorrect Solution.')
    else:
        print('\033[0;31mIncorrect Solution.')
    print('\033[0m')


Definition of the tshark parser utilities

In [3]:
from collections import namedtuple

# Definition of handshake representation
Handshake_Block = namedtuple('HandShake_Block', ('a_mac', 's_mac' ,'nonce', 'mic', 'mess', 'data', 'bytes'))

# definition of block representation
Block = namedtuple('Block', ('frame_control', 'add_1', 'add_2', 'add_3', 'qs_control', 'ccmp_par', 'data' ,'bytes'))

In [4]:
'''
=============================
* Parser auxiliar functions *
=============================
'''
def parse_handshake_block(block):
    a_mac = block[30:36]
    s_mac = block[36:42]
    nonce = block[77:109]
    mic = block[141:157]
    mess = block[60:]
    data = None if len(block) < 160 else block[159:]
    return Handshake_Block(a_mac, s_mac, nonce, mic, mess, data, block)

def parse_block_qos(block):
    frame_control = block[0:2]
    add_1 = block[4:10]
    add_2 = block[10:16]
    add_3 = block[16:22]
    qs_control = block[24:26]
    ccmp_par = (block[26:29] + chr(0x00).encode() + block[30:34])[::-1]
    data = block[34:]
    return Block(frame_control, add_1, add_2, add_3, qs_control, ccmp_par, data, block)

def parse_block(block):
    frame_control = block[0:2]
    add_1 = block[4:10]
    add_2 = block[10:16]
    add_3 = block[16:22]
    ccmp_par = (block[24:27] + chr(0x00).encode() + block[28:32])[::-1]
    data = block[32:]
    return Block(frame_control, add_1, add_2, add_3, None, ccmp_par, data, block)

Tshak parser definition

In [5]:
import subprocess
import re

class TsharkParser:

    @staticmethod
    def parse_blocks(blocks):
        parsed_blocks = []
        for block in blocks:
            # parse block line
            block_lines = ''.join([''.join(line.split()[1:]) for line in block.split('\n')])
            # cast the block to bytes
            parsed_blocks.append(bytes.fromhex(block_lines))
        return parsed_blocks
            
    @staticmethod
    def parse_output(output):
        output = output.decode()
        # detect blocks splitting by '\n' start lines
        packets_blocks = re.split(r"(?:\r?\n){2,}" ,output.strip())
        return TsharkParser.parse_blocks(packets_blocks)


Tshark utility definition

In [6]:
class Tshark:
    def __init__(self, capture_path) -> None:
        self.capture_path = capture_path
    
    def apply_filter(self, filter):
        command = f'tshark -r {self.capture_path} -Y {filter} -x --hexdump noascii'
        output = subprocess.run(command.split(), stdout=subprocess.PIPE).stdout
        return TsharkParser.parse_output(output)
    
    def find_4_handshake_sample(self):
        blocks = self.apply_filter('eapol')
        handshake_blocks = [parse_handshake_block(block) for block in blocks]
        return handshake_blocks
    
    def find_package_by_frame_number(self, frame_number):
        block = self.apply_filter(f'frame.number=={frame_number}')[0]
        # find the 802.11 package
        header_length =  int.from_bytes(block[2:4], 'little')
        block_802_11 = block[header_length:]
        # check if has QOS (if the most signinficant bit of the subtype field it's 1)
        subtype = block_802_11[:1]
        subtype = (int.from_bytes(subtype, 'little') >> 4) & 8
        if subtype == 8: #  suptype == 8 the package contains Q0S field
            return parse_block_qos(block_802_11)
        return parse_block(block_802_11)

Definition of the proccess constants 

In [7]:
SSID = 'Wifi_Test'
PASSWORD = 'Wifi_Test_Password'

# Question 1

Get the PMK that in the case of wpa2-personal is the PSK For this resason the first pass it is derive the PSK from the ssid and password using the following function:

In [8]:
def derive_psk(ssid, password):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA1(),
        salt=ssid.encode(),
        length=32,
        iterations=4096,
    )
    return kdf.derive(password.encode())

Derive the PSK and validate the solution

In [9]:
PMK = derive_psk(SSID, PASSWORD)
validate_solution(PMK.hex(), 'a22c', 'b603')

Solution: a22c4740d1843aca77252e5a6923af13773be449bb443edca5c6296ca151b603
Correct Solution.



# Question 2

Parse file and get the 4-handshake packets using the tshark class implementented

In [10]:
# init Tshark utility
tshark = Tshark('../captures/tradio.pcapng')

Get 4-handshake sample from the capture and get the nonce from the first two packages

In [11]:
# find a handshake sample
handshake_sample =  tshark.find_4_handshake_sample()

# extract the first two packages  
packet_1 = handshake_sample[0]
packet_2 = handshake_sample[1]

Extract the nonce of the first handshake packages and validate the solution

In [12]:
validate_solution(packet_1.nonce.hex(), 'b6ea', 'd7d3b')
validate_solution(packet_2.nonce.hex(), 'e6d8', 'b61c')

Solution: b6ea13c99b3a6aeeb973b622d5671d3f4a143effe67660c0bc01b774604d7d3b
Correct Solution.

Solution: e6d818b941847156d83eab024dc81f8ce372ab4a625bf0f80c383c536578b61c
Correct Solution.



# Question 3

Definition of aux function to derive the PTK 

In [13]:
def HMAC(key, algo, input):
    h = hmac.HMAC(key, algo)
    h.update(input)
    return h.finalize()

In [14]:
def SHA1_HMAC(key, input, length):
    r = bytes()
    for i in range(1 + length // 20):
        r+= HMAC(key, hashes.SHA1(), input + chr(i).encode())
    return r[:length]

Generate data blocks used to derive the PTK

In [15]:
# extract the required fields to generate the block
a_mac = packet_1.a_mac
s_mac = packet_1.s_mac
a_nonce = packet_1.nonce
s_nonce = packet_2.nonce

# ordered the fields (less first)
mac_1, mac_2 = min(a_mac, s_mac) , max(a_mac, s_mac)
nonce_1, nonce_2 = min(a_nonce, s_nonce), max(a_nonce, s_nonce)

# generate data blocks
data = 'Pairwise key expansion'.encode() + chr(0x00).encode() + mac_1 + mac_2 + nonce_1 + nonce_2

Derive the PTK key

In [16]:
PTK = SHA1_HMAC(PMK, data, 48)

Solution Validation

In [17]:
validate_solution(PTK.hex(), '0ffc', '8aad')

Solution: 0ffc1910a2ff9e32a260c3e10427ff980749105d9cfdf27cf232798126a306ef5d75e703a06de9d51b492dfdb9138aad
Correct Solution.



# Question 4

Extract aux keys from PTK

In [18]:
KCK = PTK[:16]
KEK = PTK[16:32]
TK = PTK[32:48]

In [19]:
def compute_mic(KCK, mess):
    data = mess[:81]
    for _ in range(16):
        data+=chr(0x00).encode()
    data += mess[81+16:]
    return HMAC(KCK, hashes.SHA1(), data)[:16]

To prove the authenticity of handshake messages 2, 3 and 4, the MIC is computed with the KCK and the 801.x authentication part and verified to be equal to the MIC of the packet.

In [20]:
for i in range(2, 5):
    packet = handshake_sample[i-1]
    mess = packet.mess
    mic = compute_mic(KCK, mess)
    check_equals(mess[81:81+16].hex(), mic.hex())

Expected: 27c403fe1ebc7060c97748972dc3582c
Solution: 27c403fe1ebc7060c97748972dc3582c
Correct Solution.

Expected: 35db4be1b17eab872f11066ef1f402ed
Solution: 35db4be1b17eab872f11066ef1f402ed
Correct Solution.

Expected: a29e3a96166c27cad54d30eb8644d947
Solution: a29e3a96166c27cad54d30eb8644d947
Correct Solution.



# Question 5

Find and parse the the unicast package 517

In [21]:
unicast_packet = tshark.find_package_by_frame_number(517)

Compute the nonce

In [22]:
qs_control = unicast_packet.qs_control
add_1 = unicast_packet.add_1
add_2 = unicast_packet.add_2
add_3 = unicast_packet.add_3
ccmp_par = unicast_packet.ccmp_par

nonce_unicast = qs_control[0:1] + add_2 + ccmp_par[0:2] + ccmp_par[4:8]

Solution validation

In [23]:
check_equals('002269a9e50b3500000000000b', nonce_unicast.hex())

Expected: 002269a9e50b3500000000000b
Solution: 002269a9e50b3500000000000b
Correct Solution.



# Question 6

Compute the ADD

In [24]:
frame_control = unicast_packet.frame_control
add_1 = unicast_packet.add_1
add_2 = unicast_packet.add_2
add_3 = unicast_packet.add_3
ccmp_par = unicast_packet.ccmp_par
qs_control = unicast_packet.qs_control

add_unicast = frame_control + add_1 + add_2 + add_3 + 2 * chr(0x00).encode() + qs_control[0:2]

Validate if the ADD calculated it is the solution

In [25]:
check_equals('884184aa9cfd08202269a9e50b3584aa9cfd081f00000000', add_unicast.hex())

Expected: 884184aa9cfd08202269a9e50b3584aa9cfd081f00000000
Solution: 884184aa9cfd08202269a9e50b3584aa9cfd081f00000000
Correct Solution.



# Question 7

Definition of the decrypt function

In [26]:
def AESCCM_decrypt(key, tag_length, in_data, nonce, associated_data):
    aesccm = AESCCM(key, tag_length)
    return aesccm.decrypt(nonce, in_data, associated_data)

Descrypt the unicast packet data

In [27]:
data = unicast_packet.data
plain_text = AESCCM_decrypt(TK, tag_length=8, in_data=data, nonce=nonce_unicast, associated_data=add_unicast)

Solution validation

In [28]:
validate_solution(plain_text.hex(), 'aaaa', '3637')

Solution: aaaa03000000080045000054ba9240004001fcb7c0a8010dc0a801010800d14c00020001016821640000000036110f0000000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637
Correct Solution.



# Question 8

Now we going to get the GTK that is wrapped in the third handshake packet

In [29]:
# get the handshake package that contains the GTK key
packet_3 = handshake_sample[2]

Unwrap the GTK from the handshake packet data using the aes_key_unwrap function and the KEK key

In [30]:
GTK = aes_key_unwrap(KEK, packet_3.data)

Solution validation

In [31]:
validate_solution(GTK.hex(), '3014', 'dd00')

Solution: 30140100000fac040100000fac040100000fac020c00dd16000fac010100f3b1ceb5e1d4d5e8ade18749e4cc4ec7dd00
Correct Solution.



# Question 9

Find and parse the the unicast package 527

In [32]:
multicast_packet = tshark.find_package_by_frame_number(527)

Compute the nonce of the multicast package

In [33]:
add_2 = multicast_packet.add_2
ccmp_par = multicast_packet.ccmp_par

nonce_multicast = chr(0x00).encode() + add_2 + ccmp_par[0:2] + ccmp_par[4:8]

Solution validation

In [34]:
check_equals('0084aa9cfd08200000000008f0', nonce_multicast.hex())

Expected: 0084aa9cfd08200000000008f0
Solution: 0084aa9cfd08200000000008f0
Correct Solution.



# Question 10

Compute the ADD of the multicast package

In [35]:
frame_control = multicast_packet.frame_control
add_1 = multicast_packet.add_1
add_2 = multicast_packet.add_2
add_3 = multicast_packet.add_3
ccmp_par = multicast_packet.ccmp_par
qs_control = multicast_packet.qs_control

# compute the ADD
add_multicast = frame_control + add_1 + add_2 + add_3 + 2 * chr(0x00).encode()

Solution Validation

In [36]:
check_equals('0842ffffffffffff84aa9cfd082084aa9cfd081f0000', add_multicast.hex())

Expected: 0842ffffffffffff84aa9cfd082084aa9cfd081f0000
Solution: 0842ffffffffffff84aa9cfd082084aa9cfd081f0000
Correct Solution.



# Question 11

Decrypt the data of the multicast package

In [37]:
data = multicast_packet.data
plain_text = AESCCM_decrypt(GTK[30:46], tag_length=8, in_data=data, nonce=nonce_multicast, associated_data=add_multicast)

# Question 12

In [38]:

def find_password(ssid, password_prefix, max_iterations=10):
    # init Tshark utility
    tshark = Tshark('../captures/tradio2.pcapng')
    for i in range(max_iterations):
        # generate the password
        password = f'{password_prefix}{i}'

        # Derive the psk
        pmk = derive_psk(ssid, password)
        
        # find handshake_sample
        handshake_sample =  tshark.find_4_handshake_sample()

        # extract the first two packages  
        packet_1 = handshake_sample[0]
        packet_2 = handshake_sample[1]

        # extract the required fields to generate the block
        a_mac = packet_1.a_mac
        s_mac = packet_1.s_mac
        a_nonce = packet_1.nonce
        s_nonce = packet_2.nonce

        # ordered the fields (less first)
        mac_1, mac_2 = min(a_mac, s_mac) , max(a_mac, s_mac)
        nonce_1, nonce_2 = min(a_nonce, s_nonce), max(a_nonce, s_nonce)

        # generate data blocks
        data = 'Pairwise key expansion'.encode() + chr(0x00).encode() + mac_1 + mac_2 + nonce_1 + nonce_2


        # derive PTK and subkeys
        ptk = SHA1_HMAC(pmk, data, 48)
        kck = ptk[:16]
        kek = ptk[16:32]
        tk = ptk[32:48]

        # compute the mic of the second package and check if are the same with the package mic field
        packet_2_mic = packet_2.mic
        expected_mic = compute_mic(kck, packet_2.mess)
        if packet_2_mic == expected_mic:
            # if the mic's are the same the password has been founded
            return password
    return None

Based on the knowledge of the password format, a dictionary attack is carried out where from the handshake packets it is possible to deduce the password. As a summary, if any of the tested passwords is valid, we will be able to correctly compute the MIC of a packet (for example, in this case, it is validated with the second handshake packet).

In [39]:
ssid = SSID
prefix = SSID
find_password(ssid, prefix)

'Wifi_Test2'

After applying this process, we find that the password is 'Wifi_Test2'